In [77]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import sys
import warnings

from keplergl import KeplerGl

sys.path.insert(1, '../config/')
import kepler_configs


warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

In [81]:
def func_get_Point(row):
    return shapely.geometry.Point(row.Latitude,row.Longitude)

def func_flip(x, y):
    """Flips the x and y coordinate values"""
    return y, x

In [2]:
nielson_td = pd.read_excel('../data/raw/nielson_td.xlsx',engine='openpyxl')
lahore_vor = gpd.read_file('../data/vector/dev/LAHORE/gmaps_places/lahore_voronoi_fishnet_1000m_updated.shp')

In [33]:
nielson_td = nielson_td.dropna(subset=['Latitude'])
nielson_td.reset_index(inplace=True,drop=True)
nielson_td['geometry'] = nielson_td.apply(func_get_Point,1)
nielson_td_gpd = gpd.GeoDataFrame(nielson_td,geometry='geometry')
#nielson_td_gpd.to_file('../data/nielson/nielson_td.shp')

In [64]:
nielson_td_vor = gpd.sjoin(lahore_vor,nielson_td_gpd,how='right',op='contains')
nielson_td_vor = nielson_td_vor.dropna(subset=['vor_id']).reset_index(drop=True)

In [73]:
nielson_td_vor =  nielson_td_vor[['vor_id','TDLinx','Outlet Type','Channel Band',
               'Store Monthly Turnover by Interview','Store Monthly Turnover by Obeservation',
               'Credit / Debit Card Machine','geometry']]

In [75]:
nielson_td_vor = nielson_td_vor.rename(columns={'Outlet Type':'type',
                              'Channel Band':'channel',
                              'Store Monthly Turnover by Interview':'turnover_int',
                              'Store Monthly Turnover by Obeservation':'turnover_obs',
                              'Credit / Debit Card Machine':'card_machine'})

In [82]:
nielson_td_vor['geometry'] = nielson_td_vor.geometry.apply(lambda x: shapely.ops.transform(func_flip,x))
lahore_vor['geometry'] = lahore_vor.geometry.apply(lambda x: shapely.ops.transform(func_flip,x))

In [83]:
map_ = KeplerGl(height=2000)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [84]:
map_.add_data(data=nielson_td_vor,name='neilson')
map_.add_data(data=lahore_vor,name='lahore_vor')

In [85]:
map_.save_to_html(file_name='temp.html')

Map saved to temp.html!


In [88]:
nielson_td_vor_unagg = nielson_td_vor.drop(columns=['geometry'])

In [91]:
nielson_td_vor_unagg = nielson_td_vor_unagg.merge(nielson_td[['TDLinx','Latitude','Longitude']])

In [92]:
nielson_td_vor_unagg.to_csv('../data/processed/lahore_unagg_nielson.csv')